## 

This is the promised follow-up to [Rendering a Screen Covering Triangle in OpenGL (with no buffers)](https://rauwendaal.net/2014/06/14/rendering-a-screen-covering-triangle-in-opengl/ "Rendering a Screen Covering Triangle in OpenGL (with no buffers)"), except this time the goal is to write a shader that accesses every location in a 3d texture (volume).  We use the same screen covering trick as before to draw a triangle to cover a viewport match to the X and Y dimensions of the volume, and we use instanced rendering to draw repeated triangles for each layer in the Z-dimension.

The vertex shader looks the same as before with the addition of the `instanceID`.


```{flat out int instanceID;}

void main()

{

   float x = -1.0 + float((gl_VertexID & 1) << 2);

   float y = -1.0 + float((gl_VertexID & 2) << 1);

   instanceID  = gl_InstanceID;

   gl_Position = vec4(x, y, 0, 1);
}
```


    flat out int instanceID;

    void main()

    {

       float x = -1.0 + float((gl_VertexID & 1) << 2);

       float y = -1.0 + float((gl_VertexID & 2) << 1);

       instanceID  = gl_InstanceID;

       gl_Position = vec4(x, y, 0, 1);
    }

+---------+----------------------------------------------------------+
| 1       | `flat out int` `instanceID;`                             |
|         |                                                          |
| 2       | `void` `main()`                                          |
|         |                                                          |
| 3       | `{`                                                      |
|         |                                                          |
| 4       | `float` `x = -1.0 + float((gl_VertexID & 1) << 2);`      |
|         |                                                          |
| 5       | `float` `y = -1.0 + float((gl_VertexID & 2) << 1);`      |
|         |                                                          |
| 6       | `instanceID  = gl_InstanceID;`                           |
|         |                                                          |
| 7       | `gl_Position = vec4(x, y, 0, 1);`                        |
|         |                                                          |
| 8       | `}`                                                      |
|         |                                                          |
| 9       |                                                          |
+=========+==========================================================+
|         |                                                          |
+---------+----------------------------------------------------------+

The fragment shader can then recover the voxel coordinates from `gl_FragCoord` and the `instanceID`.

+---------+----------------------------------------------------------+
| 1       | `flat in int` `instanceID;`                              |
|         |                                                          |
| 2       | `void` `main()`                                          |
|         |                                                          |
| 3       | `{`                                                      |
|         |                                                          |
| 4       | `ivec3 voxelCoord = ivec3(gl_FragCoord.xy, instanceID);` |
|         |                                                          |
| 5       | `voxelOperation(voxelCoord);`                            |
|         |                                                          |
| 6       | `}`                                                      |
|         |                                                          |
| 7       |                                                          |
+=========+==========================================================+
|         |                                                          |
+---------+----------------------------------------------------------+

Very similar to drawing the single screen covering triangle, we set our viewport to the XY-dimensions of the volume, bind a junk VAO to make certain graphics drivers happy, and call `glDrawArraysInstanced` with the Z-dimension of the volume, so that we draw a triangle per-slice of the volume.

+---------+----------------------------------------------------------+
| 1       | `glViewport(0, 0, width, height);`                       |
|         |                                                          |
| 2       | `glBindVertexArray(junkVAO);`                            |
|         |                                                          |
| 3       | `glDrawArraysInstanced(GL_TRIANGLE_STRIP, 0, 3, depth);` |
+=========+==========================================================+
|         |                                                          |
+---------+----------------------------------------------------------+

Which would look sort of like the following:

[![VolumeFillingTriangles](https://randallr.files.wordpress.com/2014/08/volumefillingtriangles.png?w=271&h=271){alt="VolumeFillingTriangles"}](https://randallr.files.wordpress.com/2014/08/volumefillingtriangles.png)

This can be useful for quickly processing a volume. Initially, I used this as an OpenGL 4.2 fallback (instead of compute shaders) so that I could still use the NSight debugger, until I realized this approach was actually outperforming the compute shader. Of course, when to use compute shaders, and how to use them effectively deserves a post of its own.